In [1]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils import data

import requests
import string
import urllib.request
import urllib.error
from pandas import *

import imageio.v3 as iio
from pathlib import Path
from tqdm import tqdm

In [2]:
from dataset import YTDataset
from cnn import Net

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cpu


In [4]:
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam, SGD

# defining the model
model = Net().to(device)
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = BCEWithLogitsLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=43200, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [5]:
## dataset
dataset_path = Path('image_data/')

train_dataset = YTDataset(dataset_path, split='train', is_norm=True)
val_dataset = YTDataset(dataset_path, split='val', is_norm=True)

batch_size = 4
num_epoch = 10

train_loader = data.DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True, drop_last=True)
val_loader = data.DataLoader(val_dataset, batch_size=1, num_workers=10, shuffle=False)

In [6]:
def train(device, model, train_loader, epoch, criterion, optimizer):
    
    print_freq = (len(train_loader.dataset) // batch_size) // 3

    model.train()

    for batch_count, (image_data, label_data, video_id) in enumerate(train_loader):

        #cuda or cpu
        image_data = image_data.to(device, dtype=torch.float)
        label_data = label_data.to(device, dtype=torch.float)
    
        
        optimizer.zero_grad()

        output_data = model(image_data)
        
        loss = criterion(output_data, label_data)

        loss.backward()

        optimizer.step()

        per_loss = loss.item()

        if batch_count%print_freq == 0:
            print('Epoch [%d/%d] Loss: %.8f' %(epoch, num_epoch, per_loss))


In [7]:
def validation(device, model, val_loader, epoch, criterion):

    model.eval()

    total_loss = 0


    with torch.no_grad():

        for batch_count, (image_data, label_data, video_id) in enumerate(tqdm(val_loader)):

            #cuda or cpu
            image_data = image_data.to(device, dtype=torch.float)
            label_data = label_data.to(device, dtype=torch.float)

            output_data = model(image_data)

            loss = criterion(output_data, label_data)
            total_loss += loss.item()

    return total_loss / len(val_loader.dataset)

In [ ]:
for epoch in range(num_epoch):

    train(device, model, train_loader, epoch, criterion, optimizer)
    loss = validation(device, model, val_loader, epoch, criterion)
    print('val_loss:' + str(loss))

Epoch [0/10] Loss: 0.73827529
Epoch [0/10] Loss: 0.74540329
Epoch [0/10] Loss: 0.68988353


100%|█████████████████████████████████████████████████████████████████████| 1076/1076 [00:57<00:00, 18.77it/s]

val_loss:0.6923979888064268


Epoch [1/10] Loss: 0.75596094
Epoch [1/10] Loss: 0.69347358
